In [1]:
import cv2
import tensorflow as tf
from keras.models import load_model
import numpy as np
from keras.preprocessing import image


faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = load_model('my_VGG16_model3.h5')

In [2]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
import cv2, pafy
url = 'https://www.youtube.com/watch?v=IZAv5Tj8a8A'
video = pafy.new(url)
print('title = ', video.title)
print('video.rating = ', video.rating)
print('video.duration = ', video.duration)
#best = video.getbest(preftype='webm')     # 'mp4','3gp'
best = video.getbest()
print('best.resolution', best.resolution)
cap=cv2.VideoCapture(best.url)

title =  [BE ORIGINAL] ITZY(있지) 'LOCO' (4K)
video.rating =  4.9664702
video.duration =  00:03:22
best.resolution 640x360


In [7]:
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("http://192.168.35.225:4747/video")

while True:
    # Capture frame-by-frame
    ret, frame =cap.read()
    
    if not ret:
        break
    
    label = ['Adult', 'Senior', 'Junior Student', 'Senior Student', 'Young Child']
    dst = cv2.resize(frame, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    dst = dst.reshape(1, 224, 224, 3)
    dst = np.array(dst, dtype=np.uint8)
    now = model.predict(dst)
    #print(now)
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    faces = faceCascade.detectMultiScale(
        rgb,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, 'AGE: {}'.format(label[now.argmax()]), (x, y-25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
     
            
            
    cv2.imshow('frame', frame)

    
    key = cv2.waitKey(25)
    if key == 27: #ESC
        break

if cap.isOpened():
    cap.release()

cv2.destroyAllWindows()